In [5]:
import docx
from docx import Document
import openpyxl as pyxl
from openpyxl import Workbook
import pandas as pd

## manual tests docx

In [6]:
with open('..\data\ER2228014 v51 ATT1 Manual As-Runs.docx', 'rb') as f:
    document = Document(f)

In [26]:
document.tables[1].cell(0,1).text

'Passed'

In [27]:
document.tables[2].cell(0,1).text

'Description'

In [32]:
statuses = []
for i, table in enumerate(document.tables):
    if table.cell(0,0).text == "Status:":
        statuses.append(table.cell(0,1).text)
    # print(table.cell(0,0).text)
    # if len(statuses) == 10:
        # break

In [33]:
print(len(statuses))
print(statuses[0:10])

135
['Passed', 'Passed', 'Passed', 'Passed', 'Passed', 'Passed', 'Passed', 'Passed', 'Passed', 'Passed']


In [46]:
test_names = []
for i, paragraph in enumerate(document.paragraphs):
    if "Run ID" in paragraph.text:
        test_names.append(document.paragraphs[i-1].text)
        # print(document.paragraphs[i-1].text)
    # print(paragraph.text)
    # if "PRD" in paragraph.text:
        # test_names.append(paragraph.text)
        # print(paragraph.text)
        # break

In [47]:
print(len(test_names))
print(test_names[0:10])

135
['PRD US13044 SRS TC12480 TC12574 Verify Subscription Message sent for Patient Equipment', 'PRD US13044 SRS TC12480 TC12575 Verify Subscription Message sent for Patient Accessories', 'PRD US13044 SRS TC12480 TC12576 Verify Subscription Message sent for Patient Task Notes', 'PRD US13044 SRS TC12480 TC12580 Verify Subscription Message sent for Patient Activity Notes', 'Toggle off - F5890', 'F6176 Patients and Tasks Reports Sticky Headers', 'PRD US32438 SRS TC12432 TC12433 TC12434 Patient Mask Suggestions', 'PRD US1554 SRS TC12438 TC12549 TC12324 TC12439 TC12440 TC12570 TC12642 TC12569 Patient Transfer To Search Location', 'F4329 - Publishing the Equipment and Accessories info in Kafka Publisher - Toggle ON', 'F4329 - EA to CO migration for  the Equipment and Accessories info in Kafka Publisher - Toggle ON']


In [49]:
df = pd.DataFrame({"test_name":test_names, "status":statuses})
df

,test_name,status
0,PRD US13044 SRS TC12480 TC12574 Verify Subscri...,Passed
1,PRD US13044 SRS TC12480 TC12575 Verify Subscri...,Passed
2,PRD US13044 SRS TC12480 TC12576 Verify Subscri...,Passed
3,PRD US13044 SRS TC12480 TC12580 Verify Subscri...,Passed
4,Toggle off - F5890,Passed
...,...,...
130,EU Toggle and Configuration Regression Test,Passed
131,UC US45400 MaskSelector tablet app integration...,Passed
132,UC US45400 MaskSelector tablet app integration...,Passed
133,UC US45400 MaskSelector tablet app integration...,Passed


In [ ]:
def load_manual_tests(file_path, return_df=True):
    # Make sure file is in docx format, not doc
    assert ".docx" in file_path, "File must be converted from .doc to .docx. Do this in Microsoft Word by selecting 'File -> Save As -> .docx'"

    # Open the file and read with python-docx package 
    with open(file_path, 'rb') as f:
        document = Document(f)

    # Get the test statuses
    statuses = []
    for i, table in enumerate(document.tables):
        if table.cell(0,0).text == "Status:":
            statuses.append(table.cell(0,1).text)
    
    # Get the test names
    test_names = []
    for i, paragraph in enumerate(document.paragraphs):
        if "Run ID" in paragraph.text:
            test_names.append(document.paragraphs[i-1].text)

    # Output as either pandas dataframe or dict, depending on return_df setting.
    data = {"test_name":test_names, "status":statuses}

    if return_df:
        return pd.DataFrame(data)
    else:
        return data

## Trace matrix stuff

In [ ]:
def load_trace(file_path, matrix_type, return_df=True):
    """Loads trace matrix .xlsx file and extracts data
    
    Args:
        file_path (String): path to the trace file
        matrix_type (String): "CO" or "PSC"
        return_df (bool, optional): If true (default), returns a pandas dataframe. Else, returns list of lists.

    Returns:
        list or pd.DataFrame: the trace data
    """
    print(f"Loading {matrix_type} from: {file_path}")

    # Some asserts to make sure inputs are valid
    assert ".xlsx" in file_path, "Trace file must be a .xlsx file"
    matrix_type = matrix_type.upper()
    assert matrix_type in ["CO", "PSC"], "Trace matrix type must be either 'CO' or 'PSC'"
    
    # Load in the actual excel file
    wb = pyxl.load_workbook(file_path)
    
    # Get the appropriate matrix from the workbook
    if matrix_type == "CO":
        ws = wb['CO Trace Matrix']
    else:
        ws = wb['PSC Trace Matrix']
        
    # get title of workbook
    title = ws['A1'].value
    print(f"Loading in worksheet titled: {title}")
    
    # extract headers for the table
    headers = [cell.value for cell in ws['A3':'E3'][0]]
    
    # load in the data row by row
    data = []
    for row in ws.iter_rows(min_row=4, max_col=5, values_only=True):
        row_data = [cell for cell in row]
        
        # Stop if all the row data is none
        # apparently the blank rows in the bottom of the worksheet are populated...
        if all([cell == None for cell in row_data]):
            break
        data.append(row_data)

    # Returns either pandas dataframe or a list of lists
    if return_df:
        return pd.DataFrame(data, columns=headers)
    else:
        return data


In [ ]:
trace_path = "..\data\ER 2228015 v42 ATT1 Sapphire 1.32 Trace Matrix.xlsx"

In [ ]:
load_trace(trace_path, "CO", return_df=True)

In [ ]:
wb = pyxl.load_workbook(trace_path)
wb.worksheets

In [ ]:
ws = wb['CO Trace Matrix']

In [ ]:
ws['A5'].value

In [ ]:
for row in ws.iter_rows(min_row=1, max_row=10, max_col=5, values_only=True):
    print(row)

In [ ]:
title = ws['A1'].value
title

In [ ]:
headers = [cell.value for cell in ws['A3':'E3'][0]]
headers

In [ ]:
data = []
for row in ws.iter_rows(min_row=4, max_col=5, values_only=True):
    row_data = [cell for cell in row]
    if all([cell == None for cell in row_data]):
        break
    data.append(row_data)

# data = [[cell for cell in row] for row in ws.iter_rows(min_row=4, max_col=5, values_only=True)]

In [ ]:
pd.DataFrame(data, columns=headers)